In [1]:
import logging
import os
import re
from kiteconnect import KiteConnect
from dotenv import load_dotenv
load_dotenv()
import bs_threading
import importlib
import time
from datetime import datetime, timedelta, timezone
import dateutil
import pandas as pd
importlib.reload(bs_threading)
from bs_threading import bs_threadify, bs_make_throttle_ready_func, bs_multiprocessify,f


In [2]:
# load all the stocks
# we do it together to the final of the sytem skk

INPUT_KITE_HISTORICAL_DIR = "input/kite_historical"
INPUT_KITE_INSTRUMENTS_DIR = "input/kite_instruments"
BACKTEST_DIR = "output/bsstonks_backtest"

In [3]:
df_all_instruments = pd.read_csv(os.path.join(INPUT_KITE_INSTRUMENTS_DIR,"instrument_list.csv"))
df_all_instruments

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,556726790,2174714,EURINR21AUG84.5000CE,EURINR,0.0,2021-08-27,84.50,0.0025,1,CE,BCD-OPT,BCD
1,556722694,2174698,EURINR21AUG84.5000PE,EURINR,0.0,2021-08-27,84.50,0.0025,1,PE,BCD-OPT,BCD
2,556833798,2175132,EURINR21AUG84.7500CE,EURINR,0.0,2021-08-27,84.75,0.0025,1,CE,BCD-OPT,BCD
3,556829958,2175117,EURINR21AUG84.7500PE,EURINR,0.0,2021-08-27,84.75,0.0025,1,PE,BCD-OPT,BCD
4,556823046,2175090,EURINR21AUG85.0000CE,EURINR,0.0,2021-08-27,85.00,0.0025,1,CE,BCD-OPT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
83599,1945089,7598,ZODJRDMKJ-BE,ZODIAC JRD MKJ-,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83600,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83601,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83602,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE


In [4]:
df_nifty = df_all_instruments[df_all_instruments["segment"].str.lower().str.startswith("indices") & df_all_instruments["name"].str.lower().str.startswith("nifty")]
df_nifty

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
15156,260617,1018,NIFTY 100,NIFTY 100,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15157,264457,1033,NIFTY 200,NIFTY 200,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15158,256265,1001,NIFTY 50,NIFTY 50,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15159,268041,1047,NIFTY 500,NIFTY 500,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15160,265993,1039,NIFTY ALPHA 50,NIFTY ALPHA 50,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15161,273673,1069,NIFTY ALPHALOWVOL,NIFTY ALPHALOWVOL,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15162,263433,1029,NIFTY AUTO,NIFTY AUTO,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15163,260105,1016,NIFTY BANK,NIFTY BANK,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15164,257289,1005,NIFTY COMMODITIES,NIFTY COMMODITIES,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE
15165,257545,1006,NIFTY CONSUMPTION,NIFTY CONSUMPTION,0.0,NaN,0.0,0.0,0,EQ,INDICES,NSE


In [5]:
for directory, _, filenames in os.walk(INPUT_KITE_HISTORICAL_DIR):
    for filename in filenames:
        print(filename)

1000193_GS170623C-GS_EQ_NSE_NSE_day.csv
1000449_PREMIERPOL_EQ_NSE_NSE_15minute.csv
1000449_PREMIERPOL_EQ_NSE_NSE_day.csv
1000705_GS230623C-GS_EQ_NSE_NSE_day.csv
1000961_GS300623C-GS_EQ_NSE_NSE_day.csv
1001729_GS020723C-GS_EQ_NSE_NSE_day.csv
1002497_GS100823C-GS_EQ_NSE_NSE_day.csv
1003009_NATCOPHARM_EQ_NSE_NSE_15minute.csv
1003009_NATCOPHARM_EQ_NSE_NSE_day.csv
1003265_GS220823C-GS_EQ_NSE_NSE_day.csv
1003777_GS090923C-GS_EQ_NSE_NSE_day.csv
1004033_GS150923C-GS_EQ_NSE_NSE_day.csv
1005569_GS200923C-GS_EQ_NSE_NSE_day.csv
1008129_INDLMETER_EQ_NSE_NSE_15minute.csv
1008129_INDLMETER_EQ_NSE_NSE_day.csv
1008385_GS101023C-GS_EQ_NSE_NSE_day.csv
1009665_GS261023C-GS_EQ_NSE_NSE_day.csv
1009921_EASUNREYRL-BZ_EQ_NSE_NSE_15minute.csv
1009921_EASUNREYRL-BZ_EQ_NSE_NSE_day.csv
1010177_GS101123C-GS_EQ_NSE_NSE_day.csv
1010945_JAKHARIA-SM_EQ_NSE_NSE_15minute.csv
1010945_JAKHARIA-SM_EQ_NSE_NSE_day.csv
101121_BEML_EQ_NSE_NSE_15minute.csv
101121_BEML_EQ_NSE_NSE_day.csv
1011457_GS071223C-GS_EQ_NSE_NSE_day.csv
10

469761_KANORICHEM_EQ_NSE_NSE_15minute.csv
469761_KANORICHEM_EQ_NSE_NSE_day.csv
470017_659RJ31-SG_EQ_NSE_NSE_day.csv
4701441_PERSISTENT_EQ_NSE_NSE_15minute.csv
4701441_PERSISTENT_EQ_NSE_NSE_day.csv
4702977_NDGL_EQ_NSE_NSE_15minute.csv
4702977_NDGL_EQ_NSE_NSE_day.csv
4704769_FRETAIL_EQ_NSE_NSE_15minute.csv
4704769_FRETAIL_EQ_NSE_NSE_day.csv
470529_KARURVYSYA_EQ_NSE_NSE_15minute.csv
470529_KARURVYSYA_EQ_NSE_NSE_day.csv
4706561_SGBFEB24-GB_EQ_NSE_NSE_15minute.csv
4706561_SGBFEB24-GB_EQ_NSE_NSE_day.csv
4706817_SGBMAR24-GB_EQ_NSE_NSE_15minute.csv
4706817_SGBMAR24-GB_EQ_NSE_NSE_day.csv
470785_653TN31-SG_EQ_NSE_NSE_day.csv
4708097_RBLBANK_EQ_NSE_NSE_15minute.csv
4708097_RBLBANK_EQ_NSE_NSE_day.csv
47105_TCPLPACK_EQ_NSE_NSE_15minute.csv
47105_TCPLPACK_EQ_NSE_NSE_day.csv
471297_KCP_EQ_NSE_NSE_15minute.csv
471297_KCP_EQ_NSE_NSE_day.csv
471553_66UP31-SG_EQ_NSE_NSE_day.csv
4716033_ASIANHOTNR_EQ_NSE_NSE_15minute.csv
4716033_ASIANHOTNR_EQ_NSE_NSE_day.csv
4717825_SGBAUG24-GB_EQ_NSE_NSE_15minute.csv
471

In [6]:
def instrument_to_file(instrument_dict, interval="day"):
    kite_instrument_token = instrument_dict['instrument_token']
    trading_symbol = instrument_dict['tradingsymbol']
    instrument_type = instrument_dict['instrument_type']
    segment = instrument_dict['segment']
    exchange = instrument_dict['exchange']
    # we've tested that _ is valid seperator by evaluating for all the instruments (86k)
    result_file = f"{kite_instrument_token}_{trading_symbol}_{instrument_type}_{segment}_{exchange}_{interval}.csv"
    return result_file




In [7]:
for idx, row in df_nifty.iterrows():
    instrument_file_name = instrument_to_file(row, interval="5minute")
    if os.path.exists(os.path.join(INPUT_KITE_HISTORICAL_DIR,instrument_file_name)):
        print(INPUT_KITE_HISTORICAL_DIR+"/"+instrument_file_name)

input/kite_historical/256265_NIFTY 50_EQ_INDICES_NSE_5minute.csv
input/kite_historical/268041_NIFTY 500_EQ_INDICES_NSE_5minute.csv


In [8]:
df_nifty_50_5min = pd.read_csv("input/kite_historical/256265_NIFTY 50_EQ_INDICES_NSE_5minute.csv")
df_nifty_50_5min
df_nifty_50_day = pd.read_csv("input/kite_historical/256265_NIFTY 50_EQ_INDICES_NSE_day.csv")
df_nifty_50_day

,date,open,high,low,close,volume,oi
0,1996-01-01 00:00:00+05:30,913.11,913.11,906.15,908.01,0,0.0
1,1996-01-02 00:00:00+05:30,906.78,906.78,899.52,900.38,0,0.0
2,1996-01-03 00:00:00+05:30,901.66,904.48,895.48,895.96,0,0.0
3,1996-01-04 00:00:00+05:30,891.37,891.37,884.10,889.67,0,0.0
4,1996-01-05 00:00:00+05:30,887.79,887.79,881.74,884.73,0,0.0
...,...,...,...,...,...,...,...
6320,2021-06-18 00:00:00+05:30,15756.50,15761.50,15450.90,15683.35,0,0.0
6321,2021-06-21 00:00:00+05:30,15525.85,15765.15,15505.65,15746.50,0,0.0
6322,2021-06-22 00:00:00+05:30,15840.50,15895.75,15752.10,15772.75,0,0.0
6323,2021-06-23 00:00:00+05:30,15862.80,15862.95,15673.95,15686.95,0,0.0


In [9]:
dt = dateutil.parser.parse("2021-06-24 00:00:00+05:30")
isttimezone = timezone(timedelta(hours=5, minutes=30))
print(dt)
print(dt.utcoffset())
print(dt.timestamp())
print(datetime(2021,6,24,tzinfo=isttimezone).timestamp())
print("current time in tz",datetime.now(timezone(timedelta(hours=1, minutes=30))))

2021-06-24 00:00:00+05:30
5:30:00
1624473000.0
1624473000.0
current time in tz 2021-06-26 14:39:24.527863+01:30


In [10]:
class INDICATORS:
    MOVING_AVERAGE = "moving_average"
    EXPONENTIAL_MOVING_AVERAGE = "exponential_moving_average"
    
    @staticmethod
    def resolve_indicator_name(indicator_name):
        # valid: moving_average, moving average, ma, MoVingAveRage
        indicator_name_fixed = re.sub("[^a-zA-Z]+", "", indicator_name)
        if indicator_name_fixed.lower() in ['movingaverage', 'ma']:
            return INDICATORS.MOVING_AVERAGE
        return indicator_name
    
    @staticmethod
    def get_moving_average_indicator(df, length=5, source='close'):
        return df[source].rolling(length).mean()
    
    @staticmethod
    def create_get_indicator_func(df):
        def get_indicator_func(indicator_name, indicator_params_dict=None):
            indicator_name = INDICATORS.resolve_indicator_name(indicator_name)
            if indicator_name in df.columns:
                return df[indicator_name]

            if indicator_name == INDICATORS.MOVING_AVERAGE:
                if indicator_params_dict == None:
                    indicator_params_dict = {}
                if 'length' not in indicator_params_dict:
                    indicator_params_dict['length'] = 5
                if 'source' not in indicator_params_dict:
                    indicator_params_dict['source'] = 'close'
                
                df_column_selector = f"{indicator_name}_{indicator_params_dict['length']}_{indicator_params_dict['source']}"
                
                if df_column_selector in df.columns:
                    return df[df_column_selector]
                else:
                    source_column = indicator_params_dict['source']
                    df[df_column_selector] = INDICATORS.get_moving_average_indicator(df, length=indicator_params_dict['length'], source=source_column)
                    return df[df_column_selector]
                
        return get_indicator_func
            
        
indicator_func = INDICATORS.create_get_indicator_func(df_nifty_50_day)
indicator_func('moving average', {'length':25})
# df_nifty_50_day

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
6320    15488.236
6321    15521.170
6322    15547.756
6323    15574.028
6324    15609.404
Name: moving_average_25_close, Length: 6325, dtype: float64

In [11]:
ma25 = indicator_func('moving average', {'length':25})

In [12]:
class BackTestOrder():
    
    def __init__(self, balance):
        self.balance = balance
        self.transactions = []
        self.symbol_stock = {}
    
    def get_balance(self, ):
        return self.balance
    
    def get_transactions(self, symbol):
#         print(type(symbol), type(self.transactions[0]['symbol']))
        return [x for x in self.transactions if x['symbol']==symbol]
    
    def get_symbol_qty(self, symbol):
        if symbol not in self.symbol_stock:
            self.symbol_stock[symbol] = 0
        return self.symbol_stock[symbol]
    
    def buy(self, symbol, date, price, qty = None, amount = None):
        if symbol not in self.symbol_stock:
            self.symbol_stock[symbol] = 0
        
        if qty == -1:
            amount = self.balance
        
        if qty is not None and qty != -1:
            amount = price * qty
            
        elif amount is not None:
            qty = amount/price
            
        self.balance -= amount
        
        if qty > 0:
            self.symbol_stock[symbol] += qty
            self.transactions.append({"symbol": symbol, "buysell": "buy", "date": date, "price":price, "qty":qty, "amount":amount, "balance":self.balance})
        
    def sell(self, symbol, date, price, qty = None, amount = None):
        if symbol not in self.symbol_stock:
            self.symbol_stock[symbol] = 0
            
        if qty == -1:
            qty = self.symbol_stock[symbol]
            
        if qty is not None and qty != -1:
            amount = price * qty
        elif amount is not None:
            qty = amount/price
        self.balance += amount
        
        if qty > 0:
            self.symbol_stock[symbol] -= qty
            self.transactions.append({"symbol": symbol, "buysell": "sell", "date": date, "price":price, "qty":qty, "amount":amount, "balance":self.balance})
            

def _run_backtest(df, order_func, start_date = None, end_date = None, seed_balance=100000):
    all_indicators = {}
    current_idx = 0
    indicator_func = INDICATORS.create_get_indicator_func(df)
    backtest_order_api = BackTestOrder(seed_balance)
    def _get_indicator(indicator_name, indicator_params_dict=None):
        nonlocal all_indicators
        nonlocal indicator_func
        indicator_ukey = f"{indicator_name}{indicator_params_dict}"
        if indicator_ukey not in all_indicators:
            all_indicators[indicator_ukey] = indicator_func(indicator_name, indicator_params_dict)
        return all_indicators[indicator_ukey].iloc[:current_idx+1]
    
    for idx, row in df.iterrows():
#         print(idx)
        if start_date is not None:
            if dateutil.parser.parse(row['date']).timestamp() < start_date.timestamp():
                continue
                
        if end_date is not None:
            if end_date.timestamp() <= dateutil.parser.parse(row['date']).timestamp():
                continue
                
        current_idx = idx
        order_func(_get_indicator, backtest_order_api)
        
#     for transaction in backtest_order_api.get_transactions(df['symbol'].iloc[-1]):
#         print(transaction)
        
    return backtest_order_api.get_transactions(df['symbol'].iloc[-1])

df_nifty = df_all_instruments[df_all_instruments["segment"].str.lower().str.startswith("indices") &
                                  df_all_instruments["name"].str.lower().str.startswith("nifty 50")]
interval = "day"

# fetch all the instruments file names
instrument_files_for_backtest = []
for idx, row in df_nifty.iterrows():
    instrument_file_name = instrument_to_file(row, interval=interval)
    if os.path.exists(os.path.join(INPUT_KITE_HISTORICAL_DIR,instrument_file_name)):
#         print(INPUT_KITE_HISTORICAL_DIR+"/"+instrument_file_name)
        instrument_files_for_backtest.append(os.path.join(INPUT_KITE_HISTORICAL_DIR, instrument_file_name))

for instrument_file in instrument_files_for_backtest:
    
    df = pd.read_csv(instrument_file)
    
    df['symbol'] = os.path.split(instrument_file)[-1].split(".")[0]
    
    print("------------------")
    print(instrument_file)
    print(df.shape)
    print("------------------")
    
    def order_func(get_indicator, backtest_order_api):
        close = get_indicator("close")
        symbol = get_indicator("symbol")
        date = get_indicator("date")
        ma15 = get_indicator("moving average", {'length':15})
        ma100 = get_indicator("moving average", {'length':100})
        if len(ma15)<2 or ma100.iloc[-2] == None:
            return
#         print(ma15.iloc[-2], ma100.iloc[-2] ,"->", ma15.iloc[-1], ma100.iloc[-1])
        if ma15.iloc[-2] <= ma100.iloc[-2] and ma15.iloc[-1] > ma100.iloc[-1]:
#             print("--------- buy ----------")
            backtest_order_api.buy(symbol.iloc[-1], date.iloc[-1], close.iloc[-2], qty=-1)
        if ma15.iloc[-2] >= ma100.iloc[-2] and ma15.iloc[-1] < ma100.iloc[-1]:
#             print("--------- sell ----------")
            backtest_order_api.sell(symbol.iloc[-1], date.iloc[-1], close.iloc[-2], qty=-1)
#         print(symbol.iloc[-1], ma.iloc[-1])
#         print(backtest_order_api.get_symbol_qty(symbol.iloc[-1]), backtest_order_api.get_balance())
    
    transactions = _run_backtest(df,order_func, start_date=datetime(2010, 1, 1), end_date = datetime(2015,1,1,tzinfo=timezone(timedelta(hours=5))), seed_balance=100000)
    print(transactions[-2:])

------------------
input/kite_historical\256265_NIFTY 50_EQ_INDICES_NSE_day.csv
(6325, 8)
------------------
[{'symbol': '256265_NIFTY 50_EQ_INDICES_NSE_day', 'buysell': 'sell', 'date': '2014-02-10 00:00:00+05:30', 'price': 6063.2, 'qty': 15.0691642060118, 'amount': 91367.35641389074, 'balance': 91367.35641389074}, {'symbol': '256265_NIFTY 50_EQ_INDICES_NSE_day', 'buysell': 'buy', 'date': '2014-03-06 00:00:00+05:30', 'price': 6328.65, 'qty': 14.437100552865262, 'amount': 91367.35641389074, 'balance': 0.0}]
------------------
input/kite_historical\268041_NIFTY 500_EQ_INDICES_NSE_day.csv
(5489, 8)
------------------
[{'symbol': '268041_NIFTY 500_EQ_INDICES_NSE_day', 'buysell': 'sell', 'date': '2014-02-11 00:00:00+05:30', 'price': 4692.55, 'qty': 21.374907607864536, 'amount': 100302.82269528473, 'balance': 100302.82269528473}, {'symbol': '268041_NIFTY 500_EQ_INDICES_NSE_day', 'buysell': 'buy', 'date': '2014-03-06 00:00:00+05:30', 'price': 4900.85, 'qty': 20.466413519141522, 'amount': 1003

In [13]:
def run_backtest(backtestname, instruments, interval, order_func, start_date = None, end_date = None, seed_balance=100000):
    backtest_dir = os.path.join(BACKTEST_DIR, backtestname)
    os.makedirs(backtest_dir, exist_ok=True)
    
    # fetch all the instruments file names
    instrument_files_for_backtest = []
    for idx, row in instruments.iterrows():
        instrument_file_name = instrument_to_file(row, interval=interval)
        if os.path.exists(os.path.join(INPUT_KITE_HISTORICAL_DIR,instrument_file_name)):
    #         print(INPUT_KITE_HISTORICAL_DIR+"/"+instrument_file_name)
            instrument_files_for_backtest.append(os.path.join(INPUT_KITE_HISTORICAL_DIR, instrument_file_name))
        
    backtest_worker_data = []
    for instrument_file in instrument_files_for_backtest:
        backtest_worker_data.append({
            'instrument_file': instrument_file,
            'backtest_dir': backtest_dir,
            'start_date': start_date,
            'end_date': end_date,
            'order_func': order_func,
            'seed_balance': seed_balance
        })
        
    def _backtest_worker_func(data):
        df = pd.read_csv(data['instrument_file'])
        df['symbol'] = os.path.split(data['instrument_file'])[-1].split(".")[0]
        
        transactions = _run_backtest(df, data['order_func'], start_date=data['start_date'], end_date = data['end_date'], seed_balance=data['seed_balance'])
        if len(transactions) > 1:
            pd.DataFrame(transactions).to_csv(os.path.join(data['backtest_dir'],df['symbol'].iloc[-1]+".csv"), index=False)
        
    bs_threadify(backtest_worker_data, _backtest_worker_func, num_threads=min(32, len(backtest_worker_data)), auto_terminate_duration=2)

In [14]:
df_all_instruments

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,556726790,2174714,EURINR21AUG84.5000CE,EURINR,0.0,2021-08-27,84.50,0.0025,1,CE,BCD-OPT,BCD
1,556722694,2174698,EURINR21AUG84.5000PE,EURINR,0.0,2021-08-27,84.50,0.0025,1,PE,BCD-OPT,BCD
2,556833798,2175132,EURINR21AUG84.7500CE,EURINR,0.0,2021-08-27,84.75,0.0025,1,CE,BCD-OPT,BCD
3,556829958,2175117,EURINR21AUG84.7500PE,EURINR,0.0,2021-08-27,84.75,0.0025,1,PE,BCD-OPT,BCD
4,556823046,2175090,EURINR21AUG85.0000CE,EURINR,0.0,2021-08-27,85.00,0.0025,1,CE,BCD-OPT,BCD
...,...,...,...,...,...,...,...,...,...,...,...,...
83599,1945089,7598,ZODJRDMKJ-BE,ZODIAC JRD MKJ-,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83600,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83601,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE
83602,979713,3827,ZUARIGLOB,ZUARI GLOBAL,0.0,NaN,0.00,0.0500,1,EQ,NSE,NSE


## Behold the 

In [ ]:



starttime = time.time()

df_nifty = df_all_instruments[df_all_instruments["segment"].str.lower().str.startswith("indices") &
                                  df_all_instruments["name"].str.lower().str.startswith("nifty")]
interval = "day"
def order_func_ma_overlap(get_indicator, backtest_order_api):
        close = get_indicator("close")
        symbol = get_indicator("symbol")
        date = get_indicator("date")
        ma15 = get_indicator("moving average", {'length':5})
        ma100 = get_indicator("moving average", {'length':100})
        if len(ma15)<2 or ma100.iloc[-2] == None:
            return
        if ma15.iloc[-2] <= ma100.iloc[-2] and ma15.iloc[-1] > ma100.iloc[-1]:
            backtest_order_api.buy(symbol.iloc[-1], date.iloc[-1], close.iloc[-2], qty=-1)
        if ma15.iloc[-2] >= ma100.iloc[-2] and ma15.iloc[-1] < ma100.iloc[-1]:
            backtest_order_api.sell(symbol.iloc[-1], date.iloc[-1], close.iloc[-2], qty=-1)
    
run_backtest("all_ma_overlap_5_100", df_all_instruments, interval, order_func_ma_overlap)
print(f"Took {time.time() - starttime:.2f}")

Exception in thread Thread-34:
Traceback (most recent call last):
  File "c:\users\nisha\miniconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "D:\AI\bsstonks\bs_threading.py", line 29, in run
    self.generic_function(data_dict)
  File "<ipython-input-13-aaae52b5d0a1>", line 28, in _backtest_worker_func
    transactions = _run_backtest(df, data['order_func'], start_date=data['start_date'], end_date = data['end_date'], seed_balance=data['seed_balance'])
  File "<ipython-input-12-cc2469443ba6>", line 86, in _run_backtest
    return backtest_order_api.get_transactions(df['symbol'].iloc[-1])
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing.py", line 895, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing.py", line 1501, in _getitem_axis
    self._validate_integer(key, axis)
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing

Exception in thread Thread-22:
Traceback (most recent call last):
  File "c:\users\nisha\miniconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "D:\AI\bsstonks\bs_threading.py", line 29, in run
    self.generic_function(data_dict)
  File "<ipython-input-13-aaae52b5d0a1>", line 28, in _backtest_worker_func
    transactions = _run_backtest(df, data['order_func'], start_date=data['start_date'], end_date = data['end_date'], seed_balance=data['seed_balance'])
  File "<ipython-input-12-cc2469443ba6>", line 86, in _run_backtest
    return backtest_order_api.get_transactions(df['symbol'].iloc[-1])
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing.py", line 895, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing.py", line 1501, in _getitem_axis
    self._validate_integer(key, axis)
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing

Exception in thread Thread-30:
Traceback (most recent call last):
  File "c:\users\nisha\miniconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "D:\AI\bsstonks\bs_threading.py", line 29, in run
    self.generic_function(data_dict)
  File "<ipython-input-13-aaae52b5d0a1>", line 28, in _backtest_worker_func
    transactions = _run_backtest(df, data['order_func'], start_date=data['start_date'], end_date = data['end_date'], seed_balance=data['seed_balance'])
  File "<ipython-input-12-cc2469443ba6>", line 86, in _run_backtest
    return backtest_order_api.get_transactions(df['symbol'].iloc[-1])
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing.py", line 895, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing.py", line 1501, in _getitem_axis
    self._validate_integer(key, axis)
  File "c:\users\nisha\miniconda3\lib\site-packages\pandas\core\indexing

## We go to next notebook where we analyze the backtest results

In [ ]:

importlib.reload(bs_threading)

# throttle_ready_func = bs_make_throttle_ready_func(min_interval_second=1.0/30.0)

def dummywork():
    print("xsdf")
    
def worker_func(q):
#     print("boss")
#         throttle_ready_func(blocking=True)
    while True:
        print("inside worker loop")
        try:
            data_dict = q.get(timeout=30)
        except:
            data_dict=None
            break
                     
        if data_dict is not None:
            try:
                print("inside worker func")
                dummywork()
                for i in range(0, 90000000):
                    x = 3
                    y = 9
                    x = x*y
                    x = x+y
                    y = x+y
            finally:
                q.task_done()
                
    
#         print("here")
# return worker_func
#     print(f'here {data}')
thedata = []
for i in range(0,15):
    thedata.append({'x':'3'})
    
for threads in [16,4, 1]:#,8,4,2,1]:
    for i in range(0,3):
        starttime = time.time()
        bs_multiprocessify([0]*300, worker_func, num_threads=threads, auto_terminate_duration=10)
        print(f"tooK : {time.time() - starttime : .2f}")


In [ ]:
import threading
threading.active_count()

In [ ]:
import multiprocessing
from multiprocessing import Process, Queue, JoinableQueue, Pool

    
with Pool(processes=4) as pool:

        # print "[0, 1, 4,..., 81]"
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)

        # evaluate "f(10)" asynchronously
        res = pool.apply_async(f, [10])
        print(res.get(timeout=1))             # prints "100"

        # make worker sleep for 10 secs
#         res = pool.apply_async(sleep, [10])


In [ ]:
def dummyfunc(num):
        print("yo")
#         time.sleep(3.0)
        print("DONE")

#         def main():
    
    
if __name__ ==  '__main__': 
    pool = multiprocessing.Pool()
    results = pool.map(dummyfunc, [1,2,3,4,5,6])
    pool.close()
    pool.join()
    print(results)

In [ ]:
multiprocessing.active_children()
multiprocessing.cpu_count()
multiprocessing.current_process()
# multiprocessing.parent_process()
